In [1]:
from GyroSolver import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%matplotlib qt

ImportError: Failed to import any of the following Qt binding modules: PyQt6, PySide6, PyQt5, PySide2

# GyroSpin : Tracé des graphes

## Partie 1 : Mouvement libre

### Grandeurs du gyroscope et conditions initiales

In [ ]:
g = 9.81           # m/s^2
m = 0.375          # kg
J3 = 0.000055      # kg.m^2
J1 = 0.000055      # kg.m^2
h = 31.25 * 1e-3   # m
x0 =  0.1
f_exc = 0.1
p = 0.1

In [ ]:
params = [g, m, h, J1, J3, x0, p, f_exc]
CI = [np.pi / 24 , 0, 0, 0, 0, 2 * np.pi * 200]

### Simulation

In [ ]:
t, the_free, phi_free, psi_free, the_d_free, phi_d_free, psi_d_free, _ = Solve_Gyro(0, 1, CI, params, solving_method='RK23', forcing='FREE', nb_points=10000)
p_the_free, p_phi_free, p_psi_free = Conjugated_Momentums(the_free, phi_free, psi_free, the_d_free, phi_d_free, psi_d_free, params)

/Users/sam-rayden/projets/GyroSpin/Article/GyroSolver.py:23: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  ans0 = odeint(dSdt_ODEINT, y0=CI, t=t, args=(g, m, h, J1, J3, x0, p, w, J1_, K, p_psi, forcing), full_output=True)


### Tracés

#### Coordonnées en fonction du temps

In [ ]:
Plot_Gyro_Angles(t, the_free, phi_free, psi_free)
Plot_Gyro_Speed_Angles(t, the_d_free, phi_d_free, psi_d_free)
Plot_Gyro_Momentum(t, p_the_free, p_phi_free, p_psi_free)

Angle ϴ : fait
Angle 𝜑 : fait
Angle 𝛙 : fait
Vitesse ϴ : fait
Vitesse 𝜑 : fait
Vitesse 𝛙 : fait
Moment ϴ : fait
Moment 𝜑 : fait
Moment 𝛙 : fait


<Figure size 640x480 with 0 Axes>

#### Pseudo-Sphère de Bloch [À FINIR]

In [ ]:
chemin = Get_Path(t, the_free, phi_free, psi_free)

In [ ]:
params_f = [g_, m_, h_, J1_, J3_, x0_, 3 *  np.pi / 2, omega_f]


T, X, Y, Z, theta, phi, psi = Get_Gyro_Position(80, 10, CI, params_f)


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set(xlim3d=(-1.2, 1.2), xlabel='X')
ax.set(ylim3d=(-1.2, 1.2), ylabel='Y')
ax.set(zlim3d=(-1.2, 1.2), zlabel='Z')
# Tracé initial du point et de la ligne
point, = ax.plot([X[0]], [Y[0]], [Z[0]], 'b.')
line, = ax.plot(X[:1], Y[:1], Z[:1], color='b')

point_static, = ax.plot(0, 0, 0, 'k.')

# Ajouter une flèche
#arrow = FancyArrowPatch((0, 0, 0), (X[0], Y[0], Z[0]), mutation_scale=15, color='blue')
#ax.add_patch(arrow)



#x_gamma_Fe = np.sin(phi + params_f[-2]) * np.sin(theta ) + X
#y_gamma_Fe = -np.cos(phi + params_f[-2]) * np.sin(theta) + Y
#z_gamma_Fe = -np.sin(theta) * np.cos(params_f[-2]) + Z

#    Ideal case
x_gamma_Fe = np.sin(params_f[-1] * T + params_f[-2]) * np.sin(theta ) + X
y_gamma_Fe = -np.cos(params_f[-1] * T + params_f[-2]) * np.sin(theta) + Y
z_gamma_Fe = -np.sin(theta) * np.cos(params_f[-2] + params_f[-1] * T - phi) + Z

x_gamma_P = np.cos(phi) * np.sin(theta) + X * 0.5
y_gamma_P = np.sin(phi) * np.sin(theta) + Y * 0.5
z_gamma_P = Z * 0.5



# Initialiser la flèche
arrow = ax.quiver(0, 0, 0, 0, 0, 0, color='blue', arrow_length_ratio=0.01)

torque_Fe = ax.quiver(0, 0, 0, 0, 0, 0, color='red', arrow_length_ratio=0.01)

torque_P = ax.quiver(0, 0, 0, 0, 0, 0, color='green', arrow_length_ratio=100)


text_time = ax.text2D(0.02, 0.95, '', transform=ax.transAxes, color='black')

text_CI = ax.text2D(0.02, 1, r'$\theta_0 =$' + str(round(CI[0] * 180 / np.pi, 0)) + r'$^\circ$', transform=ax.transAxes, color='black')
text_CI_omega = ax.text2D(0.02, 0.9, r'$f_{\psi} =$' + str(round(CI[-1] / (2 * np.pi), 0)) + '$\;\mathrm{Hz}$', transform=ax.transAxes, color='black')


# Fonction de mise à jour pour l'animation
def update(num, X, Y, Z, point, line, point_static, arrow):
    point.set_data(X[num-1:num], Y[num-1:num])
    point.set_3d_properties(Z[num-1:num])

    line.set_data(X[:num], Y[:num])
    line.set_3d_properties(Z[:num])

    # Mettre à jour la position de la flèche
    arrow.set_segments([[[0, 0, 0], [X[num], Y[num], Z[num]]]])

    torque_Fe.set_segments([[[X[num], Y[num], Z[num]], [x_gamma_Fe[num], y_gamma_Fe[num], z_gamma_Fe[num]]]])
    torque_P.set_segments([[[0.5 * X[num], 0.5 * Y[num], 0.5 * Z[num]], [x_gamma_P[num], y_gamma_P[num], z_gamma_P[num]]]])

    text_time.set_text(f'Time: {T[num]:.2f} s')


    return point, line, point_static, arrow

# Augmenter la fréquence de trame pour accélérer l'animation
frame_rate = 10
num_frames = len(T)
interval = 50 #1000 / frame_rate

# Animer le tracé
ani = FuncAnimation(fig, update, frames=num_frames, interval=interval,
                    fargs=(X, Y, Z, point, line, point_static, arrow), blit=False)

#### Moment des forces

In [ ]:
moment_poids = Momentum_Weight(the_free, phi_free, psi_free, params)

In [ ]:
plt.figure(figsize=[3, 3], dpi = 300)
plt.xlabel(r'$t [\text{s}]$')
plt.ylabel(r'$ \Vert\vec{\Gamma}(\vec{P}) \Vert   \quad [mgh]$')
plt.grid()
plt.savefig("./Graphes/GyroFree-MomentPoids.png", bbox_inches="tight")
plt.close()
print("Moment du poids : fait")

Moment du poids : fait
